In [1]:
### IMPORTS ###
!pip install mediapipe
!pip install opencv

import cv2
import mediapipe as mp
import numpy as np
import time
import os
import sys
import argparse
import json
import requests
import datetime
import threading




[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

# Function to calculate angle between three points
def calculate_angle(v1, v0, v2):
    v1 = np.array(v1)
    v0 = np.array(v0)
    v2 = np.array(v2)
    
    angle_1 = (np.arctan2(v2[1]-v0[1], v2[0]-v0[0]) - np.arctan2(v1[1]-v0[1], v1[0]-v0[0]))
    angle_2 = np.arccos(np.dot(v0-v1, v2-v0) / (np.linalg.norm(v0-v1) * np.linalg.norm(v2-v0)))
    angle = np.abs(angle_1 * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

status = None
count = 0
reps = 10
side = "left"
total_sets = 3
sets = 0
# Main loop
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            # Extract landmarks for right shoulder, elbow, and wrist
            landmarks = results.pose_landmarks.landmark
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate angle between the shoulder, elbow, and wrist
            angle = calculate_angle(shoulder, elbow, wrist)
            shoulder_angle = calculate_angle(hip, shoulder, elbow)
            
            # Draw semi-circle at elbow
            cv2.ellipse(image, tuple(np.multiply(elbow, [640, 480]).astype(int)), (80, 80), 0, 0, -(int(angle)), (255, 0, 0), 2)
            
            # Draw circle spot on shoulder and elbow
            
            # Check if arms are raised above threshold angle (e.g., 120 degrees)
            if shoulder_angle > 100:
                cv2.putText(image, f"Please lower your elbow!", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.circle(image, tuple(np.multiply(shoulder, [640, 480]).astype(int)), 10, (0, 0, 255), -1)  # Red spot on shoulder
                
            elif shoulder_angle < 70:
                cv2.putText(image, f"Please raise your elbow!", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.circle(image, tuple(np.multiply(shoulder, [640, 480]).astype(int)), 10, (0, 0, 255), -1)  # Red spot on shoulder
            else:
                cv2.circle(image, tuple(np.multiply(shoulder, [640, 480]).astype(int)), 10, (0, 255, 0), -1)  # Green spot on shoulder
                
                
            
            threshold_angle = 120
            # Calculate proportion of current angle to threshold angle
            proportion = max(0, min(1, angle / threshold_angle))
            # Calculate radius of filled circle based on proportion
            filled_circle_radius = int(proportion * threshold_angle / 4)

            # Draw hollow circle at elbow with maximum radius
            cv2.circle(image, tuple(np.multiply(elbow, [640, 480]).astype(int)), int(threshold_angle / 4), (255, 255, 255), 2)

            # Draw filled circle at elbow with radius proportional to the angle
            if int(threshold_angle / 4)-5 < filled_circle_radius < int(threshold_angle / 4)+5:
                cv2.circle(image, tuple(np.multiply(elbow, [640, 480]).astype(int)), filled_circle_radius, (0, 255, 0), -1)
            elif filled_circle_radius > int(threshold_angle / 4):
                cv2.circle(image, tuple(np.multiply(elbow, [640, 480]).astype(int)), filled_circle_radius, (0, 0, 255), -1)
            else:
                cv2.circle(image, tuple(np.multiply(elbow, [640, 480]).astype(int)), filled_circle_radius, (255, 0, 0), -1)
            

            if angle >= threshold_angle and angle <threshold_angle+10:
                if status == "Lower":
                    count += 1
                status = 'Raise'
                # Reset filled circle when threshold angle is met
                filled_circle_radius = 0
            elif angle > threshold_angle+10:
                cv2.putText(image, "Do not over extend!", (250, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.circle(image, tuple(np.multiply(elbow, [640, 480]).astype(int)), circle_radius, (0, 0, 255), -1)
            elif angle < 50:
                status = 'Lower'
            
            if status == 'Raise':
                cv2.putText(image, "Contract!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            elif status == 'Lower':
                cv2.putText(image, "Extend!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        except:
            pass
        circle_radius = int(angle/4)
        #cv2.circle(image, tuple(np.multiply(elbow, [640, 480]).astype(int)), circle_radius, (255, 0, 0), -1)  # Red spot on elbow
        
        cv2.putText(image,f"Count: {count}", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)   
        # Draw pose landmarks
        # if results.pose_landmarks:
        #     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        #                               mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
        #                               mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))
        
        # Display frame
        if count == reps:
            sets+=1
            if sets == total_sets:
                cv2.putText(image, "Workout Complete!", (200, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                break
            else:
                cv2.putText(image, "Set Complete! Time to switch sides!", (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                if side == "left":
                    side = "right"
                elif side == "right":
                    side = "left"
                status = None
                count = 0
        cv2.imshow('Mediapipe Feed', image)
        
        # Exit loop if 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    # Release video capture and close all windows
    cap.release()
    cv2.destroyAllWindows()
